In [1]:
!pip install numpy transformers pathlib pandas gdown lxml hf_transfer

  Using cached pathlib-1.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached hf_transfer-0.1.9-cp38-abi3-win_amd64.whl.metadata (1.8 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -------------------- ------------------- 2.1/4.0 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 11.0 MB/s eta 0:00:00
Using cached hf_transfer-0.1.9-cp38-abi3-win_amd64.whl (1.2 MB)
Using cached beautifulsoup4-4.14.2-py3-none-any.whl (106 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)


In [2]:
!gdown --folder "https://drive.google.com/drive/folders/1wAS0umYohuR53r4sqroxxiG2ab5p5msn"

^C


Processing file 1dw1yoa2ZbdCBNOY4rPNKwS2fZfUR_O3F fine_tuning.csv
Processing file 1lrRuZCk4xERL8CbeD2c5B_0v0fNY00o_ sample_submission.csv
Processing file 1nZ9YFe5dVR_Ipq-LpD8hmtZ9OnODcyz- test.csv


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1dw1yoa2ZbdCBNOY4rPNKwS2fZfUR_O3F
From (redirected): https://drive.google.com/uc?id=1dw1yoa2ZbdCBNOY4rPNKwS2fZfUR_O3F&confirm=t&uuid=b5943bc8-601c-42b5-b18d-f1786326527b
To: f:\AI-Contest\Genomic_Language_Model\genomic_language_model\fine_tuning.csv

  0%|          | 0.00/409M [00:00<?, ?B/s]
  0%|          | 524k/409M [00:00<02:39, 2.56MB/s]
  1%|          | 2.10M/409M [00:00<01:00, 6.76MB/s]
  1%|          | 3.67M/409M [00:00<00:46, 8.79MB/s]
  1%|▏         | 5.24M/409M [00:00<00:40, 9.89MB/s]
  2%|▏         | 6.82M/409M [00:00<00:38, 10.6MB/s]
  2%|▏         | 8.39M/409M [00:00<00:36, 11.0MB/s]
  2%|▏         | 9.96M/409M [00:01<00:35, 11.2MB/s]
  3%|▎         | 11.5M/409M [00:01<00:34, 11.4MB/s]
  3%|▎         | 13.1M/409M [00:01<00:34, 11.6MB/s]
  4%|▎         | 14.7M/409M [00:01<00:33, 11

In [3]:
import os
import math
import random
import numpy as np
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForMaskedLM

c:\Users\PNC\anaconda3\envs\Mol_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

data_path = './Data/'
df = pd.read_csv(data_path + 'test.csv')

# 데이터 최대 길이 확인
max_seq_len = df["seq"].str.len().max()
print(f"✅ Rows = {len(df):,}, Max sequence length = {max_seq_len}")

✅ Rows = 13,711, Max sequence length = 1024


In [6]:
BATCH_SIZE = 64
NUM_WORKERS = 2

MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-100m-multi-species"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)
model = model.to(DEVICE).eval()

MODEL_CAP = tokenizer.model_max_length 
EFFECTIVE_MAX_LEN = min(MODEL_CAP, max_seq_len)

In [7]:
print(MODEL_CAP, max_seq_len)

2048 1024


In [8]:
model

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(4107, 512, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(2050, 512, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-21): 22 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((512,), eps=1e-12, elementwi

In [9]:
tokenizer

EsmTokenizer(name_or_path='InstaDeepAI/nucleotide-transformer-v2-100m-multi-species', vocab_size=4107, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
EFFECTIVE_MAX_LEN

1024

In [11]:
class SeqDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.ids  = df["ID"].tolist()
        self.seqs = df["seq"].tolist()
        self.tok  = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return {"ID": self.ids[idx], "seq": self.seqs[idx]}

def collate_fn(batch, tok=tokenizer, max_len=EFFECTIVE_MAX_LEN):
    ids  = [b["ID"] for b in batch]
    seqs = [b["seq"] for b in batch]
    enc  = tok.batch_encode_plus(
        seqs,
        return_tensors="pt",
        padding="longest",          
        truncation=True,
        max_length=max_len
    )
    # attention_mask: pad 토큰이 0
    return {
        "ids": ids,
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"]
    }

In [12]:
dataset = SeqDataset(df, tokenizer, EFFECTIVE_MAX_LEN)
loader  = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False,
                     num_workers=NUM_WORKERS, collate_fn=collate_fn)
print("✅ Dataloader ready.")

✅ Dataloader ready.


In [ ]:
all_ids = []
all_embs = []
use_amp = (DEVICE == "cuda")

with torch.no_grad():
    for batch in loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attn_mask = batch["attention_mask"].to(DEVICE)

        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            outs = model(
                input_ids,
                attention_mask=attn_mask,
                encoder_attention_mask=attn_mask,   
                output_hidden_states=True,
            )
            # 마지막 히든스테이트: (B, L, H)
            last_hidden = outs.hidden_states[-1]    # torch.Tensor

            # mask 모양 맞추기: (B, L, 1)
            mask_exp = attn_mask.unsqueeze(-1)      # 1 for valid tokens

            # 패딩 제외 평균: sum(hidden * mask) / sum(mask)
            summed = (last_hidden * mask_exp).sum(dim=1)                    # (B, H)
            counts = mask_exp.sum(dim=1).clamp(min=1)                       # (B, 1)
            seq_emb = summed / counts                                       # (B, H)

        all_ids.extend(batch["ids"])
        all_embs.append(seq_emb.detach().cpu())

emb = torch.vstack(all_embs).float()        # (N, H)
N, H = emb.shape
print(f"✅ Embedding shape = {N} x {H}")

In [ ]:
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')

emb_np = emb.numpy()
emb_cols = [f"emb_{i:04d}" for i in range(emb_np.shape[1])]
emb_df = pd.DataFrame(emb_np, columns=emb_cols)

submission = pd.concat([sample_submission['ID'], emb_df], axis=1)
submission.to_csv('baseline_submission.csv', index=False)